# Requirements

In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
!pip install polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 13.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import polars as pl

In [4]:
import re

# Reading Data

In [5]:
dft = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet')

# Analysis

In [6]:
pl.read_parquet_schema('/kaggle/input/aeroclub-recsys-2025/train.parquet')

Schema([('Id', Int64),
        ('bySelf', Boolean),
        ('companyID', Int64),
        ('corporateTariffCode', Int64),
        ('frequentFlyer', String),
        ('nationality', Int64),
        ('isAccess3D', Boolean),
        ('isVip', Boolean),
        ('legs0_arrivalAt', String),
        ('legs0_departureAt', String),
        ('legs0_duration', String),
        ('legs0_segments0_aircraft_code', String),
        ('legs0_segments0_arrivalTo_airport_city_iata', String),
        ('legs0_segments0_arrivalTo_airport_iata', String),
        ('legs0_segments0_baggageAllowance_quantity', Float64),
        ('legs0_segments0_baggageAllowance_weightMeasurementType', Float64),
        ('legs0_segments0_cabinClass', Float64),
        ('legs0_segments0_departureFrom_airport_iata', String),
        ('legs0_segments0_duration', String),
        ('legs0_segments0_flightNumber', String),
        ('legs0_segments0_marketingCarrier_code', String),
        ('legs0_segments0_operatingCarrier_code', Str

In [7]:
dft

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64
0,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T16:20:00""","""2024-06-15T15:40:00""","""02:40:00""","""YK2""","""KJA""","""KJA""",1.0,0.0,1.0,"""TLK""","""02:40:00""","""216""","""KV""","""KV""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,370.0,16884.0,1,0
1,true,57323,123,"""S7/SU/UT""",36,true,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,2240.0,51125.0,0,1
2,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,2087645,"""98ce0dabf696

# Cleaning Data & Feature Engineering

In [8]:
dft= dft.drop('bySelf','sex','legs0_segments0_cabinClass','nationality','legs0_segments0_marketingCarrier_code','legs0_segments2_marketingCarrier_code','legs0_segments3_marketingCarrier_code','legs1_segments0_marketingCarrier_code','legs1_segments2_marketingCarrier_code','legs1_segments3_marketingCarrier_code')

In [9]:
dft = dft.drop('legs1_segments0_duration')

In [10]:
dft = dft.with_columns(
    TotalPrice = pl.col("totalPrice") + pl.col("taxes"))

In [11]:
dft['TotalPrice']

TotalPrice
f64
17254.0
53365.0
55935.0
84120.0
88310.0
…
36290.0
33220.0
29920.0


In [12]:
dft = dft.drop('legs0_segments0_duration','legs0_segments1_duration','legs0_segments2_duration','legs0_segments3_duration')

In [13]:
dft = dft.drop('legs1_segments1_duration','legs1_segments2_duration','legs1_segments3_duration')

In [14]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
str,str,str,str,str,str
"""2024-06-15T15:40:00""","""2024-06-15T16:20:00""","""02:40:00""","""2024-07-09T09:45:00""","""2024-07-09T14:20:00""","""02:35:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
…,…,…,…,…,…
"""2024-11-05T20:50:00""","""2024-11-06T09:45:00""","""10:55:00""","""2024-11-09T19:25:00""","""2024-11-10T08:10:00""","""14:45:00"""
"""2024-11-05T00:20:00""","""2024-11-05T20:00:00""","""17:40:00""","""2024-11-09T21:10:00""","""2024-11-10T08:35:00""","""13:25:00"""
"""2024-11-05T00:20:00""","""2024-11-05T20:00:00""","""17:40:00""","""2024-11-09T19:25:00""","""2024-11-10T08:10:00""","""14:45:00"""


In [15]:
dft= dft.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration').str.replace(r"T", "  "))

In [16]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
str,str,str,str,str,str
"""2024-06-15 15:40:00""","""2024-06-15 16:20:00""","""02:40:00""","""2024-07-09 09:45:00""","""2024-07-09 14:20:00""","""02:35:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
…,…,…,…,…,…
"""2024-11-05 20:50:00""","""2024-11-06 09:45:00""","""10:55:00""","""2024-11-09 19:25:00""","""2024-11-10 08:10:00""","""14:45:00"""
"""2024-11-05 00:20:00""","""2024-11-05 20:00:00""","""17:40:00""","""2024-11-09 21:10:00""","""2024-11-10 08:35:00""","""13:25:00"""
"""2024-11-05 00:20:00""","""2024-11-05 20:00:00""","""17:40:00""","""2024-11-09 19:25:00""","""2024-11-10 08:10:00""","""14:45:00"""


In [17]:
dft= dft.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs1_departureAt',
'legs1_arrivalAt').str.to_datetime("%Y-%m-%d %H:%M:%S"))

In [18]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs1_departureAt',
'legs1_arrivalAt']

legs0_departureAt,legs0_arrivalAt,legs1_departureAt,legs1_arrivalAt
datetime[μs],datetime[μs],datetime[μs],datetime[μs]
2024-06-15 15:40:00,2024-06-15 16:20:00,2024-07-09 09:45:00,2024-07-09 14:20:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
…,…,…,…
2024-11-05 20:50:00,2024-11-06 09:45:00,2024-11-09 19:25:00,2024-11-10 08:10:00
2024-11-05 00:20:00,2024-11-05 20:00:00,2024-11-09 21:10:00,2024-11-10 08:35:00
2024-11-05 00:20:00,2024-11-05 20:00:00,2024-11-09 19:25:00,2024-11-10 08:10:00


In [19]:
dft['legs0_duration',
'legs1_duration']

legs0_duration,legs1_duration
str,str
"""02:40:00""","""02:35:00"""
"""07:25:00""","""08:25:00"""
"""07:25:00""","""08:25:00"""
"""07:25:00""","""08:25:00"""
"""07:25:00""","""08:25:00"""
…,…
"""10:55:00""","""14:45:00"""
"""17:40:00""","""13:25:00"""
"""17:40:00""","""14:45:00"""


In [20]:
dft = dft.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.replace_all(r"\.\d{2}", "")
)

In [21]:
dft=dft.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.to_time("%H:%M:%S"))

In [22]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
datetime[μs],datetime[μs],time,datetime[μs],datetime[μs],time
2024-06-15 15:40:00,2024-06-15 16:20:00,02:40:00,2024-07-09 09:45:00,2024-07-09 14:20:00,02:35:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
…,…,…,…,…,…
2024-11-05 20:50:00,2024-11-06 09:45:00,10:55:00,2024-11-09 19:25:00,2024-11-10 08:10:00,14:45:00
2024-11-05 00:20:00,2024-11-05 20:00:00,17:40:00,2024-11-09 21:10:00,2024-11-10 08:35:00,13:25:00
2024-11-05 00:20:00,2024-11-05 20:00:00,17:40:00,2024-11-09 19:25:00,2024-11-10 08:10:00,14:45:00


In [23]:
dftt = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/test.parquet')

In [24]:
dftt= dftt.drop('bySelf','sex','legs0_segments0_cabinClass','nationality','legs0_segments0_marketingCarrier_code','legs0_segments2_marketingCarrier_code','legs0_segments3_marketingCarrier_code','legs1_segments0_marketingCarrier_code','legs1_segments2_marketingCarrier_code','legs1_segments3_marketingCarrier_code',)

In [25]:
dftt = dftt.with_columns(
    TotalPrice = pl.col("totalPrice") + pl.col("taxes"))

In [26]:
dftt = dftt.drop('legs1_segments0_duration')

In [27]:
dftt = dftt.drop('legs0_segments0_duration','legs0_segments1_duration','legs0_segments2_duration','legs0_segments3_duration')

In [28]:
dftt = dftt.drop('legs1_segments1_duration','legs1_segments2_duration','legs1_segments3_duration')

In [29]:
dftt['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
str,str,str,str,str,str
"""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""02:30:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""02:45:00"""
"""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""02:30:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""02:45:00"""
"""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""02:30:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""02:45:00"""
"""2024-12-19T08:25:00""","""2024-12-19T12:45:00""","""02:20:00""","""2024-12-21T12:00:00""","""2024-12-21T12:35:00""","""02:35:00"""
"""2024-12-19T08:25:00""","""2024-12-19T12:45:00""","""02:20:00""","""2024-12-21T12:00:00""","""2024-12-21T12:35:00""","""02:35:00"""
…,…,…,…,…,…
"""2025-01-08T09:05:00""","""2025-01-08T12:50:00""","""02:45:00""",null,null,null
"""2025-01-08T21:25:00""","""2025-01-09T01:10:00""","""02:45:00""",null,null,null
"""2025-01-08T21:25:00""","""2025-01-09T01:10:00""","""02:45:00""",null,null,null


In [30]:
dftt= dftt.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration').str.replace(r"T", "  "))

In [31]:
dftt= dftt.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs1_departureAt',
'legs1_arrivalAt').str.to_datetime("%Y-%m-%d %H:%M:%S"))

In [32]:
dftt = dftt.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.replace_all(r"\.\d{2}", "")
)

In [33]:
dftt=dftt.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.to_time("%H:%M:%S"))

In [34]:
dftt['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
datetime[μs],datetime[μs],time,datetime[μs],datetime[μs],time
2024-12-19 06:50:00,2024-12-19 11:20:00,02:30:00,2024-12-21 21:10:00,2024-12-21 21:55:00,02:45:00
2024-12-19 06:50:00,2024-12-19 11:20:00,02:30:00,2024-12-21 21:10:00,2024-12-21 21:55:00,02:45:00
2024-12-19 06:50:00,2024-12-19 11:20:00,02:30:00,2024-12-21 21:10:00,2024-12-21 21:55:00,02:45:00
2024-12-19 08:25:00,2024-12-19 12:45:00,02:20:00,2024-12-21 12:00:00,2024-12-21 12:35:00,02:35:00
2024-12-19 08:25:00,2024-12-19 12:45:00,02:20:00,2024-12-21 12:00:00,2024-12-21 12:35:00,02:35:00
…,…,…,…,…,…
2025-01-08 09:05:00,2025-01-08 12:50:00,02:45:00,null,null,null
2025-01-08 21:25:00,2025-01-09 01:10:00,02:45:00,null,null,null
2025-01-08 21:25:00,2025-01-09 01:10:00,02:45:00,null,null,null


In [35]:
def preprocess(df: pl.DataFrame) -> pl.DataFrame:
    datetime_cols = ["legs0_departureAt", "legs0_arrivalAt",
                     "legs1_departureAt", "legs1_arrivalAt"]

    for col in datetime_cols:
        # Ensure datetime type
        df = df.with_columns(
            pl.col(col).cast(pl.Datetime(time_unit="us"))
        )

        # Basic datetime features
        df = df.with_columns([
            pl.col(col).dt.hour().alias(f"{col}_hour"),
            pl.col(col).dt.weekday().alias(f"{col}_dow"),
            pl.col(col).dt.month().alias(f"{col}_month"),
            (pl.col(col).dt.weekday() >= 5).cast(pl.Int8).alias(f"{col}_is_weekend"),
        ])

        # Cyclical hour encoding
        df = df.with_columns([
            (pl.col(f"{col}_hour") * (2 * np.pi) / 24).sin().alias(f"{col}_hour_sin"),
            (pl.col(f"{col}_hour") * (2 * np.pi) / 24).cos().alias(f"{col}_hour_cos"),
        ])

    # Duration to minutes
    duration_cols = ["legs0_duration", "legs1_duration"]
    for col in duration_cols:
    # If already a Duration, just convert to minutes
        if df[col].dtype == pl.Duration(time_unit="us"):
            df = df.with_columns(
                (pl.col(col).dt.seconds() / 60).alias(col)
            )
        # If it's a Time (HH:MM:SS)
        elif df[col].dtype == pl.Time:
            df = df.with_columns(
                (pl.col(col).dt.hour() * 60 +
                 pl.col(col).dt.minute() +
                 pl.col(col).dt.second() / 60).alias(col)
            )
        # If it's a string like "02:40:00"
        else:
            df = df.with_columns(
                (pl.col(col).str.strptime(pl.Time, "%H:%M:%S", strict=False).dt.hour() * 60 +
                 pl.col(col).str.strptime(pl.Time, "%H:%M:%S", strict=False).dt.minute() +
                 pl.col(col).str.strptime(pl.Time, "%H:%M:%S", strict=False).dt.second() / 60
                ).alias(col)
            )
   
    return df

In [36]:
print("Preprocessing...")
dft = preprocess(dft)
dftt = preprocess(dftt)

Preprocessing...


# Feature Selection

In [37]:
x = dft['Id','TotalPrice','searchRoute','corporateTariffCode','profileId','companyID',
'legs0_duration' ,
'legs1_duration','legs0_segments0_seatsAvailable','legs0_segments1_seatsAvailable','legs1_segments0_seatsAvailable','legs1_segments1_seatsAvailable',
'miniRules0_monetaryAmount' ,
'miniRules0_percentage',
'miniRules0_statusInfos' ,
'miniRules1_monetaryAmount' ,
'miniRules1_percentage' ,
'miniRules1_statusInfos' ,
'pricingInfo_isAccessTP' ,
'pricingInfo_passengerCount','legs0_segments0_baggageAllowance_quantity','legs0_segments1_baggageAllowance_quantity','legs1_segments0_baggageAllowance_quantity','legs1_segments1_baggageAllowance_quantity',
'legs0_segments0_arrivalTo_airport_iata','legs0_segments1_arrivalTo_airport_iata',
'legs1_segments0_arrivalTo_airport_iata','legs1_segments1_arrivalTo_airport_iata',
'legs0_segments0_arrivalTo_airport_city_iata','legs0_segments1_arrivalTo_airport_city_iata',
'legs1_segments0_arrivalTo_airport_city_iata','legs1_segments1_arrivalTo_airport_city_iata']

In [38]:
string_columns = [col_name for col_name, dtype in zip(x.columns, x.dtypes) if dtype == pl.Utf8]
print(string_columns)

['searchRoute', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments1_arrivalTo_airport_iata', 'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs1_segments0_arrivalTo_airport_city_iata', 'legs1_segments1_arrivalTo_airport_city_iata']


In [39]:
CATEGORICAL_COLS = ['searchRoute', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments1_arrivalTo_airport_iata',  'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs1_segments0_arrivalTo_airport_city_iata', 'legs1_segments1_arrivalTo_airport_city_iata']


In [40]:
xt = dftt['Id','TotalPrice','searchRoute','corporateTariffCode','profileId','companyID',
'legs0_duration' ,
'legs1_duration','legs0_segments0_seatsAvailable','legs0_segments1_seatsAvailable','legs1_segments0_seatsAvailable','legs1_segments1_seatsAvailable',
'miniRules0_monetaryAmount' ,
'miniRules0_percentage',
'miniRules0_statusInfos' ,
'miniRules1_monetaryAmount' ,
'miniRules1_percentage' ,
'miniRules1_statusInfos' ,
'pricingInfo_isAccessTP' ,
'pricingInfo_passengerCount','legs0_segments0_baggageAllowance_quantity','legs0_segments1_baggageAllowance_quantity','legs1_segments0_baggageAllowance_quantity','legs1_segments1_baggageAllowance_quantity',
'legs0_segments0_arrivalTo_airport_iata','legs0_segments1_arrivalTo_airport_iata',
'legs1_segments0_arrivalTo_airport_iata','legs1_segments1_arrivalTo_airport_iata',
'legs0_segments0_arrivalTo_airport_city_iata','legs0_segments1_arrivalTo_airport_city_iata',
'legs1_segments0_arrivalTo_airport_city_iata','legs1_segments1_arrivalTo_airport_city_iata']

In [41]:
dft['legs0_departureAt',
 'legs0_arrivalAt',
 'legs0_duration',
 'legs1_departureAt',
 'legs1_arrivalAt',
 'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
datetime[μs],datetime[μs],f64,datetime[μs],datetime[μs],f64
2024-06-15 15:40:00,2024-06-15 16:20:00,-96.0,2024-07-09 09:45:00,2024-07-09 14:20:00,-101.0
2024-06-15 09:25:00,2024-06-15 14:50:00,-67.0,2024-07-09 22:05:00,2024-07-10 08:30:00,-7.0
2024-06-15 09:25:00,2024-06-15 14:50:00,-67.0,2024-07-09 22:05:00,2024-07-10 08:30:00,-7.0
2024-06-15 09:25:00,2024-06-15 14:50:00,-67.0,2024-07-09 22:05:00,2024-07-10 08:30:00,-7.0
2024-06-15 09:25:00,2024-06-15 14:50:00,-67.0,2024-07-09 22:05:00,2024-07-10 08:30:00,-7.0
…,…,…,…,…,…
2024-11-05 20:50:00,2024-11-06 09:45:00,-113.0,2024-11-09 19:25:00,2024-11-10 08:10:00,117.0
2024-11-05 00:20:00,2024-11-05 20:00:00,36.0,2024-11-09 21:10:00,2024-11-10 08:35:00,37.0
2024-11-05 00:20:00,2024-11-05 20:00:00,36.0,2024-11-09 19:25:00,2024-11-10 08:10:00,117.0


In [42]:
dft = dft.with_columns(
    pl.col("legs1_duration","legs0_duration").abs()
)

In [43]:
dftt = dftt.with_columns(
    pl.col("legs1_duration","legs0_duration").abs()
)

In [44]:
x

Id,TotalPrice,searchRoute,corporateTariffCode,profileId,companyID,legs0_duration,legs1_duration,legs0_segments0_seatsAvailable,legs0_segments1_seatsAvailable,legs1_segments0_seatsAvailable,legs1_segments1_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,legs0_segments0_baggageAllowance_quantity,legs0_segments1_baggageAllowance_quantity,legs1_segments0_baggageAllowance_quantity,legs1_segments1_baggageAllowance_quantity,legs0_segments0_arrivalTo_airport_iata,legs0_segments1_arrivalTo_airport_iata,legs1_segments0_arrivalTo_airport_iata,legs1_segments1_arrivalTo_airport_iata,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_city_iata,legs1_segments0_arrivalTo_airport_city_iata,legs1_segments1_arrivalTo_airport_city_iata
i64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,str,str,str,str,str,str,str,str
0,17254.0,"""TLKKJA/KJATLK""",null,2087645,57323,-96.0,-101.0,9.0,null,9.0,null,null,null,null,null,null,null,1.0,1,1.0,null,1.0,null,"""KJA""",null,"""TLK""",null,"""KJA""",null,"""TLK""",null
1,53365.0,"""TLKKJA/KJATLK""",123,2087645,57323,-67.0,-7.0,4.0,4.0,9.0,9.0,2300.0,null,1.0,3500.0,null,1.0,1.0,1,1.0,1.0,1.0,1.0,"""OVB""","""KJA""","""OVB""","""TLK""","""OVB""","""KJA""","""OVB""","""TLK"""
2,55935.0,"""TLKKJA/KJATLK""",null,2087645,57323,-67.0,-7.0,4.0,4.0,9.0,9.0,2300.0,null,1.0,3500.0,null,1.0,1.0,1,1.0,1.0,1.0,1.0,"""OVB""","""KJA""","""OVB""","""TLK""","""OVB""","""KJA""","""OVB""","""TLK"""
3,84120.0,"""TLKKJA/KJATLK""",123,2087645,57323,-67.0,-7.0,4.0,4.0,9.0,9.0,0.0,null,1.0,0.0,null,1.0,1.0,1,1.0,1.0,1.0,1.0,"""OVB""","""KJA""","""OVB""","""TLK""","""OVB""","""KJA""","""OVB""","""TLK"""
4,88310.0,"""TLKKJA/KJATLK""",null,2087645,57323,-67.0,-7.0,4.0,4.0,9.0,9.0,0.0,null,1.0,0.0,null,1.0,1.0,1,1.0,1.0,1.0,1.0,"""OVB""","""KJA""","""OVB""","""TLK""","""OVB""","""KJA""","""OVB""","""TLK"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
18146427,36290.0,"""MOWSVX/SVXMOW""",44,3046852,54154,-113.0,117.0,3.0,5.0,8.0,1.0,24000.0,null,1.0,0.0,null,0.0,0.0,1,0.0,0.0,0.0,0.0,"""SGC""","""SVX""","""UFA""","""VKO""","""SGC""","""SVX""","""UFA""","""MOW"""
18146428,33220.0,"""MOWSVX/SVXMOW""",44,3046852,54154,36.0,37.0,3.0,9.0,4.0,3.0,24000.0,null,1.0,0.0,null,0.0,0.0,1,0.0,0.0,0.0,0.0,"""UFA""","""SVX""","""SGC""","""VKO""","""UFA""","""SVX""","""SGC""","""MOW"""
18146429,29920.0,"""MOWSVX/SVXMOW""",44,3046852,54154,36.0,117.0,3.0,9.0,8.0,1.0,24000.0,null,1.0,0.0,null,0.0,0.0,1,0.0,0.0,0.0,0.0,"""UFA""","""SVX""","""UFA""","""VKO""","""UFA""","""SVX""","""UFA""","""MOW"""


In [45]:
x = x.with_columns([
    pl.col(c).cast(pl.Categorical).to_physical().alias(c)
    for c in x.columns if x.schema[c] == pl.Utf8
])


In [46]:
xt = xt.with_columns([
    pl.col(c).cast(pl.Categorical).to_physical().alias(c)
    for c in xt.columns if xt.schema[c] == pl.Utf8
])


In [47]:
x

Id,TotalPrice,searchRoute,corporateTariffCode,profileId,companyID,legs0_duration,legs1_duration,legs0_segments0_seatsAvailable,legs0_segments1_seatsAvailable,legs1_segments0_seatsAvailable,legs1_segments1_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,legs0_segments0_baggageAllowance_quantity,legs0_segments1_baggageAllowance_quantity,legs1_segments0_baggageAllowance_quantity,legs1_segments1_baggageAllowance_quantity,legs0_segments0_arrivalTo_airport_iata,legs0_segments1_arrivalTo_airport_iata,legs1_segments0_arrivalTo_airport_iata,legs1_segments1_arrivalTo_airport_iata,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_city_iata,legs1_segments0_arrivalTo_airport_city_iata,legs1_segments1_arrivalTo_airport_city_iata
i64,f64,u32,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,u32,u32,u32,u32,u32,u32,u32,u32
0,17254.0,0,null,2087645,57323,-96.0,-101.0,9.0,null,9.0,null,null,null,null,null,null,null,1.0,1,1.0,null,1.0,null,77,null,56,null,77,null,56,null
1,53365.0,0,123,2087645,57323,-67.0,-7.0,4.0,4.0,9.0,9.0,2300.0,null,1.0,3500.0,null,1.0,1.0,1,1.0,1.0,1.0,1.0,63,77,63,56,63,77,63,56
2,55935.0,0,null,2087645,57323,-67.0,-7.0,4.0,4.0,9.0,9.0,2300.0,null,1.0,3500.0,null,1.0,1.0,1,1.0,1.0,1.0,1.0,63,77,63,56,63,77,63,56
3,84120.0,0,123,2087645,57323,-67.0,-7.0,4.0,4.0,9.0,9.0,0.0,null,1.0,0.0,null,1.0,1.0,1,1.0,1.0,1.0,1.0,63,77,63,56,63,77,63,56
4,88310.0,0,null,2087645,57323,-67.0,-7.0,4.0,4.0,9.0,9.0,0.0,null,1.0,0.0,null,1.0,1.0,1,1.0,1.0,1.0,1.0,63,77,63,56,63,77,63,56
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
18146427,36290.0,225,44,3046852,54154,-113.0,117.0,3.0,5.0,8.0,1.0,24000.0,null,1.0,0.0,null,0.0,0.0,1,0.0,0.0,0.0,0.0,100,78,66,81,100,78,66,154
18146428,33220.0,225,44,3046852,54154,36.0,37.0,3.0,9.0,4.0,3.0,24000.0,null,1.0,0.0,null,0.0,0.0,1,0.0,0.0,0.0,0.0,66,78,100,81,66,78,100,154
18146429,29920.0,225,44,3046852,54154,36.0,117.0,3.0,9.0,8.0,1.0,24000.0,null,1.0,0.0,null,0.0,0.0,1,0.0,0.0,0.0,0.0,66,78,66,81,66,78,66,154


In [48]:
y = dft['selected']

In [49]:
y

selected
i64
1
0
0
0
0
…
0
0
0


In [50]:
x = x.drop('miniRules0_percentage','miniRules1_percentage')

In [51]:
xt = xt.drop('miniRules0_percentage','miniRules1_percentage')

# Trainig Model

In [52]:
!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 21.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [53]:
import lightgbm as lgb

In [54]:
X_train = x.to_numpy()
y_train = y.to_numpy()

X_test = xt.to_numpy()


In [55]:
model = lgb.LGBMRegressor()
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.744954 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3818
[LightGBM] [Info] Number of data points in the train set: 18145372, number of used features: 30
[LightGBM] [Info] Start training from score 0.005816


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [56]:
import joblib
#Save the model
joblib.dump(model, '/kaggle/working/my_lgb_model.pkl')

['/kaggle/working/my_lgb_model.pkl']

In [57]:
# Predict the Test set results
y_pred = model.predict(X_test)

/usr/local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [58]:
dftt = dftt.with_columns(pl.Series("score", y_pred))

In [59]:
submission = (
    dftt
    .with_columns(
        pl.struct(["score", "Id"])
        .rank(method="ordinal", descending=True)
        .over("ranker_id")
        .cast(pl.Int64)
        .alias("selected")
    )
    .select(["Id", "ranker_id", "selected"])
)

submission.write_csv("/kaggle/working/lgb_model.csv")

In [60]:
import  pandas as pd
z = pd.read_csv('/kaggle/working/lgb_model.csv')
z.shape
z.head(20)

,Id,ranker_id,selected
0,18144679,c9373e5f772e43d593dd6ad2fa90f67a,144
1,18144680,c9373e5f772e43d593dd6ad2fa90f67a,64
2,18144681,c9373e5f772e43d593dd6ad2fa90f67a,304
3,18144682,c9373e5f772e43d593dd6ad2fa90f67a,242
4,18144683,c9373e5f772e43d593dd6ad2fa90f67a,193
5,18144684,c9373e5f772e43d593dd6ad2fa90f67a,291
6,18144685,c9373e5f772e43d593dd6ad2fa90f67a,408
7,18144686,c9373e5f772e43d593dd6ad2fa90f67a,143
8,18144687,c9373e5f772e43d593dd6ad2fa90f67a,63
9,18144688,c9373e5f772e43d593dd6ad2fa90f67a,303
